In [1]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate



In [2]:
# 데이터 읽어오기
import json

game_sentences_file = "./용례_게임.json"

with open(game_sentences_file, encoding='utf-8', mode='r') as f:
    data = json.load(f)
    data = list(map(lambda x:{"sentence":x["sentence"], "facet":x["tokens"][0]["facet"]}, data))
    
    
df = pd.DataFrame(data)
df


,sentence,facet
0,"퀘스트 사냥꾼은 대체로 컨트롤 성향 덱에 더 강한 모습을 보이나, 컨트롤 성기사엔 ...",직업
1,"나이트 페이 성약의 단 이동 연결망 1단계를 강화하면 그루터, 숲의 끝자락, 고요한...",소속
2,딜칭호는 아마 설 즈음에 풀릴거 같구요,아이템
3,2년 전 네로제에서 켈트사제를 때려잡았던 것처럼 2년 후 세시키의 활약을 기대해보겠...,캐릭터
4,그럴 경우 토마가 보호막 + 불원소 부여 역할로 쓸만하지 않을까 싶습니다.,시스템용어
...,...,...
199499,- 죽사르가 욕 제일먹은 이유중 하나가,아이템
199500,그래서 그 컨셉대로 동료와 조우하는 임무는 무시하고 관우 혼자 단신으로 형주를 뚫는...,온라인
199501,야타도 이 파티로 패고 오는데 바위공명 쩔긴 쩝니다.,시스템용어
199502,이노시스 오세요 특히 데브펜이 매우 사기 좋네요 제발 사줘,온라인


In [33]:
# 데이터 프레임 내보내기

df.to_csv('game_procceed.csv', index=False)

In [4]:
df['facet'].unique()

array(['직업', '소속', '아이템', '캐릭터', '시스템용어', '기술', '몬스터', '임무', '오브젝트',
       '온라인', 'NPC', '고유명', '역할', '종족', '계급', '동료', '시즌', '오프라인', '조작도구'],
      dtype=object)

In [5]:
# 데이터 전처리
import re

## 결측치 제거
df = df.dropna()

## sentence에서 특수문자, 개행문자 제거

def clean_text(sent):
    sent_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", " ", sent) # 특수문자 제거
    return sent_clean

df['sentence'] = df['sentence'].apply(lambda x: clean_text(x))

df

,sentence,facet
0,퀘스트 사냥꾼은 대체로 컨트롤 성향 덱에 더 강한 모습을 보이나 컨트롤 성기사엔 ...,직업
1,나이트 페이 성약의 단 이동 연결망 단계를 강화하면 그루터 숲의 끝자락 고요한...,소속
2,딜칭호는 아마 설 즈음에 풀릴거 같구요,아이템
3,년 전 네로제에서 켈트사제를 때려잡았던 것처럼 년 후 세시키의 활약을 기대해보겠...,캐릭터
4,그럴 경우 토마가 보호막 불원소 부여 역할로 쓸만하지 않을까 싶습니다,시스템용어
...,...,...
199499,죽사르가 욕 제일먹은 이유중 하나가,아이템
199500,그래서 그 컨셉대로 동료와 조우하는 임무는 무시하고 관우 혼자 단신으로 형주를 뚫는...,온라인
199501,야타도 이 파티로 패고 오는데 바위공명 쩔긴 쩝니다,시스템용어
199502,이노시스 오세요 특히 데브펜이 매우 사기 좋네요 제발 사줘,온라인


In [8]:
# 데이터 분리

X = df['sentence']
y = df['facet']

In [15]:
# 단어 인덱스 시퀀스 벡터

corpus = [preprocessing.text.text_to_word_sequence(text) for text in X] # 단어 시퀀스 벡터
tokenizer = preprocessing.text.Tokenizer() 
tokenizer.fit_on_texts(corpus) # tokenizer 수행
sequences = tokenizer.texts_to_sequences(corpus) # 단어 시퀀스 벡터를 수치화된 벡터로 변환
word_index = tokenizer.word_index # 단어 인덱스

MAX_SEQ_LEN = 15 # 단어 시퀀스 벡터 크기 -> 15자 미만의 문장은 패딩 처리, 15자가 넘는 문장은 절단
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post') 

padded_seqs[0]

array([ 25859,    621,      2,    768,    653,  23265,   1291, 161591,
          170,  97692,   3445, 161592,   1357,   4823,   3753],
      dtype=int32)

In [16]:
# 하이퍼파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(word_index) + 1 # 전체 단어 수
# CNN 모델 정의
input_layer = Input(shape=(MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

In [17]:
conv1 = Conv1D(filters=128, kernel_size=3, padding='valid', activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)
conv2 = Conv1D(filters=128, kernel_size=4, padding='valid', activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)
conv3 = Conv1D(filters=128, kernel_size=5, padding='valid', activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)
# 합치기
concat = concatenate([pool1, pool2, pool3])

In [29]:
hidden = Dense(128, activation=tf.nn.relu)(concat) 
dropout_hidden = Dropout(rate=dropout_prob)(hidden) 
logits = Dense(len(y.unique()), name='logits')(dropout_hidden)

In [30]:
predictions = Dense(len(y.unique()), activation=tf.nn.softmax)(logits)

In [31]:
# 학습용, 검증용, 테스트용 데이터셋 생성
# 학습셋:검증셋:테스트셋 = 7:2:1

from sklearn.preprocessing import LabelEncoder

labels = y.values
# 문자로 된 label을 숫자로 변환
encoder = LabelEncoder()
encoder.fit(labels)
labels = encoder.transform(labels)

ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(df))
train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)
train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

In [32]:
# 모델 생성
model = Model(inputs=input_layer, outputs=predictions) 
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Epoch 1/5


2024-01-29 17:27:16.363354: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [199504]
	 [[{{node Placeholder/_1}}]]
2024-01-29 17:27:16.363566: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [199504]
	 [[{{node Placeholder/_1}}]]


 790/6983 [==>...........................] - ETA: 24:06 - loss: 2.3120 - accuracy: 0.2077

KeyboardInterrupt: 

In [37]:
encoder.inverse_transform([x for x in range(len(y.unique()))])

array(['NPC', '계급', '고유명', '기술', '동료', '몬스터', '소속', '시스템용어', '시즌', '아이템',
       '역할', '오브젝트', '오프라인', '온라인', '임무', '조작도구', '종족', '직업', '캐릭터'],
      dtype=object)